In [208]:
import pandas as pd
import numpy as np
import datetime

In [219]:
def convert_price(str_price):
    if type(str_price) != str:
        return str_price
    if ('－' in str_price) | ('なし' in str_price):
        return 0.0
    elif '万円' in str_price:
        return float(str_price.replace('万円', '').replace(',', ''))*10000
    else:
        return float(str_price.replace('円', '').replace(',', ''))
    
    
def convert_area(str_area):
    if type(str_area) != str:
        return str_area
    if ('－' in str_area) | ('なし' in str_area):
        return 0.0
    elif ',' in str_area:
        #これは異常値なのでNaNを返す
        return np.nan
    else:
        return float(str_area.replace('m²', '').replace('（壁芯）','').replace('（内法）', ''))
    
def convert_age(str_age):
    if type(str_age) != str:
        return str_age
    pos_y = str_age.find('年')
    year = int(str_age[0:pos_y])
    pos_m = str_age.find('月')
    month = int(str_age[pos_y+1:pos_m])
    
    return (datetime.date.today().year - year)*12 + datetime.date.today().month - month

def convert_tosu(str_tosu):
    if type(str_tosu) != str:
        return str_tosu
    if str_tosu == '－':
        #これは欠損値なのでとりあえずNaNを返す
        return np.nan
    return int(str_tosu.replace('戸', '').replace(',', ''))

def convert_floor(str_floor):
    if type(str_floor) != str:
        return str_floor
    if str_floor == '－':
        # これは欠損値
        return np.nan
    pos_floor = str_floor.find('階')
    # 地下室は負の値にする
    return int(str_floor[0:pos_floor].replace('メゾネット', '').replace('地下','-'))
    

def convert_total_floor(str_floor):
    if type(str_floor) != str:
        return str_floor
    if str_floor == '－':
        # これは欠損値
        return np.nan
    pos_total_floor = str_floor.find('階')
    return int(str_floor[0:pos_total_floor].replace('地上', ''))

In [220]:
df_data_ori = pd.read_csv('./bunjou_2018-4-28.csv')

In [221]:
delete_column = ['タイトル', 'その他交通', '所在地', '借地期間・地代（月額）', '権利金', '敷金 / 保証金', '維持費等',
                'その他一時金', '建物状況調査　/　瑕疵保険・保証', '備考', 'リフォーム履歴', 'リノベーション履歴', 'バイク置き場',
                 '駐輪場', 'ペット', '敷地面積', '管理形態・方式', '国土法届出', '条件等', '現況', '引渡し', '物件番号',
                 '情報公開日', '次回更新予定日']

In [222]:
df_data_ori.columns

Index(['タイトル', '交通', 'その他交通', '所在地', '物件種目', '価格', '平米単価', '管理費等', '修繕積立金',
       '借地期間・地代（月額）', '権利金', '敷金 / 保証金', '維持費等', 'その他一時金', ' ', '建物名・部屋番号',
       '設備', '建物状況調査　/　瑕疵保険・保証', '備考', '間取り', '専有面積', 'バルコニー', '階建 / 階',
       '建物構造', '築年月', '総戸数', 'リフォーム履歴', 'リノベーション履歴', '駐車場', 'バイク置き場', '駐輪場',
       'ペット', '土地権利', '敷地面積', '管理形態・方式', '国土法届出', '条件等', '現況', '引渡し', '物件番号',
       '情報公開日', '次回更新予定日'],
      dtype='object')

In [223]:
df_kari = df_data_ori
df_kari = df_kari.drop(delete_column, axis=1)

In [224]:
price_column = ['価格', '平米単価', '管理費等', '修繕積立金']
area_column = ['専有面積', 'バルコニー']
for key in price_column:
    df_kari[key] = df_kari[key].map(convert_price)
for key in area_column:
    df_kari[key] = df_kari[key].map(convert_area)
    
df_kari['築年月'] = df_kari['築年月'].map(convert_age)
df_kari['総戸数'] = df_kari['総戸数'].map(convert_tosu)
df_floor = df_kari['階建 / 階'].str.split(' /', expand=True)
df_floor.columns = ['総階数', '階']
df_floor['総階数'] = df_floor['総階数'].map(convert_total_floor)
df_floor['階'] = df_floor['階'].map(convert_floor)

df_kari = pd.concat([df_kari, df_floor], axis=1)
df_kari = df_kari.drop('階建 / 階', axis=1)

abnormal_data = []    
for i in range(0, len(df_kari.index)):
    for key in (price_column + area_column):
        if df_kari[key][i] == np.nan:
            abnormal_data.append(i)
print(abnormal_data)
#df_kari = df_kari.drop(abnormal_data, axis=0).reset_index(drop=True)

[]


In [225]:
opts = {}
for i in range(0, len(df_kari['設備'])):
    #if df_kari['設備'][i]:
    setubi = df_kari['設備'][i].split("、")
    for j in range(0, len(setubi)):
        if setubi[j] in opts:
            opts[setubi[j]] += 1
        else:
            opts[setubi[j]] = 1
            
pd.DataFrame.from_dict(opts, orient='index').sort_values(by=0, ascending=False) 

,0
エレベーター,3028
システムキッチン,2772
都市ガス,2402
浴室乾燥機,2383
追焚機能,2279
温水洗浄便座,2279
室内洗濯機置場,2249
オートロック,2051
収納スペース,1994
給湯,1989


In [226]:
df_kari.reset_index()

,index,交通,物件種目,価格,平米単価,管理費等,修繕積立金,,建物名・部屋番号,設備,間取り,専有面積,バルコニー,建物構造,築年月,総戸数,駐車場,土地権利,総階数,階
0,0,日暮里舎人ライナー / 江北駅 徒歩3分 （電車ルート案内）,中古マンション,15800000.0,272600.0,9540.0,8200.0,,西新井スカイハイツ,システムキッチン、全居室収納、室内洗濯機置場、クローゼット、モニター付インターホン、３口以上...,２ＬＤＫ,57.98,6.72,ＲＣ,467,133.0,－,所有権,6,3
1,1,都営三田線 / 高島平駅 徒歩4分 （電車ルート案内）,中古マンション,17500000.0,313500.0,8000.0,10000.0,,高島平第３住宅 ８号棟,バイク置き場、収納スペース、敷地内公園、エレベーター、オートバス、給湯、駐輪場、照明器具、ケ...,３ＤＫ,55.83,9.10,ＳＲＣ,550,110.0,"有 17,000円/月",所有権,11,2
2,2,日暮里舎人ライナー / 江北駅 徒歩4分 （電車ルート案内）,中古マンション,17800000.0,289000.0,10012.0,7400.0,,西新井大師扇スカイハイツ,システムキッチン、室内洗濯機置場、クローゼット、モニター付インターホン、ディンプルキー、防犯...,３ＬＤＫ,61.60,7.60,ＳＲＣ,453,302.0,空無,所有権,11,10
3,3,日暮里舎人ライナー / 江北駅 徒歩1分 （電車ルート案内）,中古マンション,17800000.0,263400.0,9900.0,16980.0,,ソフィア西新井,－,３ＳＬＤＫ,67.60,0.00,ＲＣ,422,101.0,空無,所有権,7,4
4,4,都営三田線 / 志村三丁目駅 徒歩5分 （電車ルート案内）,中古マンション,17800000.0,310000.0,10500.0,20000.0,,志村カーネルマンション,システムキッチン、収納スペース、室内洗濯機置場、モニター付インターホン、３口以上コンロ、エレ...,２ＬＤＫ(ＬＤＫ１１．７洋室６．６洋室７．４),57.42,2.60,ＳＲＣ,529,22.0,無,所有権,8,3
5,5,東京メトロ有楽町線 / 地下鉄成増駅 徒歩4分 （電車ルート案内）,中古マンション,18000000.0,327300.0,16765.0,15260.0,,サングローリー成増,システムキッチン、クローゼット、３口以上コンロ、追焚機能、シャワー付洗面化粧台、温水洗浄便座,３ＤＫ,55.00,5.58,ＲＣ,307,21.0,－,旧法賃借権,5,2
6,6,東京メトロ副都心線 / 地下鉄成増駅 徒歩4分 （電車ルート案内）,中古マンション,18900000.0,343700.0,16765.0,15260.0,,サングローリー成増,－,２ＬＤＫ,55.00,5.58,ＲＣ,306,21.0,無,旧法賃借権,5,4
7,7,東京メトロ日比谷線 / 三ノ輪駅 徒歩11分 （電車ルート案内）,中古マンション,19800000.0,319600.0,12270.0,18590.0,,ライオンズマンション三河島,システムキッチン、シューズインクローゼット、モニター付インターホン、エレベーター、都市ガス、...,２ＤＫ,61.97,6.65,ＳＲＣ,443,28.0,－,所有権,10,2
8,8,京成本線 / 京成小岩駅 徒歩1分 （電車ルート案内）,中古マンション,19800000.0,329000.0,10000.0,16900.0,,グリーンコーポ京成小岩,室内洗濯機置場、エレベーター、給湯、都市ガス、二面バルコニー,３ＤＫ,60.20,10.36,ＳＲＣ,462,43.0,無,所有権,9,2
9,9,京成本線 / 京成小岩駅 徒歩1分 （電車ルート案内）,中古マンション,19800000.0,329000.0,10000.0,16900.0,,グリーンコーポ京成小岩,追焚機能、都市ガス、ルーフバルコニー,３ＤＫ,60.20,10.36,ＳＲＣ,462,40.0,無,所有権,9,2
